# Pruebas para la generacion de datos de entrenamiento

## Cambios para que funcione con clases

Para exportar se ve que crear una clase para almacenar todas las percepciones es muy util por lo que se procede a modificar todas las funciones anteriores que lo requieran para que funcionen con la clase de percepciones.

#### Librerias

In [35]:
import os
import math
import pickle
from zmqRemoteApi import RemoteAPIClient
from random import randint as ri
from random import uniform as ru
import pandas as pd

#### Creacion de clases

In [2]:
#Class for simulation object
class sim_objects ():
    def __init__(self):
        self.client = RemoteAPIClient()
        self.sim = self.client.getObject('sim')
        self.file = 0
        self.executed = False
        self.joint_handler_ids = []
        self.num_joints = 0
        self.obj_handler_ids = []
        self.sequencies = 1
        self.seq_steps = 2
        self.training_data = []
        
sim_obj = sim_objects()

# Class to store the different handlers the robot needs to operate in the simulator. 
class joint_handlers_class ():
    def __init__(self):
        self.JRF0 = 0
        self.JRF1 = 0
        self.JRB0 = 0
        self.JRB1 = 0
        self.JLF0 = 0
        self.JLF1 = 0
        self.JLB0 = 0
        self.JLB1 = 0
        
handler = joint_handlers_class()

class obj_handlers_class ():
    def __init__(self):
        self.JRF2 = 0
        self.JRB2 = 0
        self.JLF2 = 0
        self.JLB2 = 0
        self.base = 0

obj_handler = obj_handlers_class()

#### Funciones de carga del quad y generacion de acciones random

In [89]:
def init_position():
    start_pos = ri(0,1)
    #Fixes the last joints position to 0 so they wont move
    for i in sim_obj.obj_handler_ids[:-1]:
        sim_obj.sim.setJointTargetPosition(i, 0)
    if start_pos == 0:
        print("Flat")
        #Set all the joints to their position
        for i in sim_obj.joint_handler_ids:
            sim_obj.sim.setJointTargetPosition(i, 0)
        #After setting all the joints it moves    
        for i in range(20):
            sim_obj.client.step()
    else:
        sim_obj.client.step()
        print("Standing")
    
            

In [4]:
# Function that load the robot in the scene at the beggining of each evaluation.
def load_quad_class():
    # Opening the connection with CoppeliaSim simulator
    sim_obj.client = RemoteAPIClient()
    sim_obj.sim = sim_obj.client.getObject('sim')
    
    # Setting the simulation mode for working in sinchronous mode. The simulation timming is controlled by the Python program and not by the simulation itselfs.
    sim_obj.client.setStepping(True)
    
    #Right-front
    handler.JRF0 = sim_obj.sim.getObject("/JRF0")
    handler.JRF1 = sim_obj.sim.getObject("/JRF1")
    #Right-back
    handler.JRB0 = sim_obj.sim.getObject("/JRB0")
    handler.JRB1 = sim_obj.sim.getObject("/JRB1")
    #Left-front
    handler.JLF0 = sim_obj.sim.getObject("/JLF0")
    handler.JLF1 = sim_obj.sim.getObject("/JLF1")
    #Left-back
    handler.JLB0 = sim_obj.sim.getObject("/JLB0")
    handler.JLB1 = sim_obj.sim.getObject("/JLB1")

    #Setting obj handlers
    obj_handler.base = sim_obj.sim.getObject("/Base")
    obj_handler.JRF2 = sim_obj.sim.getObject("/JRF2")
    obj_handler.JRB2 = sim_obj.sim.getObject("/JRB2")
    obj_handler.JLF2 = sim_obj.sim.getObject("/JLF2")
    obj_handler.JLB2 = sim_obj.sim.getObject("/JLB2")

    
    #Stores the different id for every handler starting with the base
    sim_obj.joint_handler_ids = list(handler.__dict__.values())

    sim_obj.num_joints = len(sim_obj.joint_handler_ids)

    sim_obj.obj_handler_ids = list(obj_handler.__dict__.values())


    sim_obj.sim.startSimulation()

#### Clase y funcion para percepciones

In [27]:

#Class to store all the perceptions
class perceptions ():
    def __init__(self):
        #Execution data
        self.sequence = 0
        self.step = 0
        self.increments = []

        #Initial perceptions
        self.prev_j_positions = []
        self.prev_j_velocities = []
        self.prev_j_forces = []
        self.prev_base_pos_x = 0
        self.prev_base_pos_y = 0
        self.prev_base_pos_z = 0
        self.prev_base_ori_alpha = 0
        self.prev_base_ori_beta = 0
        self.prev_base_ori_gamma = 0
        #self.prev_base_accel = []

        #Resulting perceptions
        self.post_j_positions = []
        self.post_j_velocities = []
        self.post_j_forces = []
        self.post_base_pos_x = 0
        self.post_base_pos_y = 0
        self.post_base_pos_z = 0
        self.post_base_ori_alpha = 0
        self.post_base_ori_beta = 0
        self.post_base_ori_gamma = 0
        #self.post_base_accel = []

perception = perceptions()

#### Pruebas para guardar las percepciones por paso

In [92]:
def get_joints_preceptions():

    #Base perceptions
    base_x, base_y, base_z = sim_obj.sim.getObjectPosition(obj_handler.base, sim_obj.sim.handle_world)
    base_alpha, base_beta, base_gamma = sim_obj.sim.getObjectOrientation(obj_handler.base, sim_obj.sim.handle_world)
    if  not sim_obj.executed:
        perception.prev_base_pos_x = base_x
        perception.prev_base_pos_y = base_y
        perception.prev_base_pos_z = base_z
        perception.prev_base_ori_alpha = base_alpha
        perception.prev_base_ori_beta = base_beta
        perception.prev_base_ori_gamma = base_gamma
    else:
        perception.post_base_pos_x = base_x
        perception.post_base_pos_y = base_y
        perception.post_base_pos_z = base_z
        perception.post_base_ori_alpha = base_alpha
        perception.post_base_ori_beta = base_beta
        perception.post_base_ori_gamma = base_gamma
        
    #Joint perceptions
    for i in sim_obj.joint_handler_ids:
        #Angular/linear position of the joint
        pos = sim_obj.sim.getJointPosition(i)
        #Torque/force being aplied by the joint
        force = sim_obj.sim.getJointForce(i)
        #Angular/linear velocity of the joint's movement
        veloc = sim_obj.sim.getJointVelocity(i)
        if  not sim_obj.executed:
            perception.prev_j_positions.append(pos)
            perception.prev_j_velocities.append(veloc)
            perception.prev_j_forces.append(force)
        else:
            perception.post_j_positions.append(pos)
            perception.post_j_velocities.append(veloc)
            perception.post_j_forces.append(force)

In [51]:
def move_joints():
    for i in range(sim_obj.num_joints):
        sim_obj.sim.setJointTargetPosition(sim_obj.joint_handler_ids[i], (perception.increments[i]+perception.prev_j_positions[i]))
    #After setting all the joints it moves
    for i in range(5):
        sim_obj.client.step()

In [52]:
#Generate list of random positions
def rand_gen():
    for i in range(sim_obj.num_joints):
        n = ru(-10, 10) * math.pi / 180
        perception.increments.append(n)

#### Exportar

In [65]:
def export_pickle():    
    #Dump data in file
    sim_obj.file = open(os.getcwd() + r'\training_dataset.txt', 'wb')
    pickle.dump(sim_obj.training_data, sim_obj.file)
    sim_obj.file.close()

#### Ejecucion

##### Multiples secuencias y multiples pasos

##### Pruebas finales con el main

In [85]:
sim_obj.sequencies = 4
print("Numero de secuencias: ",sim_obj.sequencies)
sim_obj.seq_steps = 3
print("Steps por secuencia: ",sim_obj.seq_steps)
sim_obj.training_data = []

Numero de secuencias:  4
Steps por secuencia:  3


In [90]:
#Main function for multiple executions and multiple steps
#def main():
print("main function")
sim_obj.training_data = []
for k in range(sim_obj.sequencies):
    load_quad_class()
    init_position()
    for j in range(sim_obj.seq_steps):
        perception = perceptions()
        perception.sequence = k
        perception.step = j         
        print("----------- Secuencia: ", perception.sequence)
        print("----------- Paso: ", perception.step)
        rand_gen()
        get_joints_preceptions()
        #Execute every joint action
        move_joints()
        sim_obj.executed = True
        get_joints_preceptions()
        sim_obj.executed = False
        sim_obj.training_data.append(vars(perception))
    sim_obj.sim.stopSimulation()
#export()

main function
Flat
----------- Secuencia:  0
----------- Paso:  0
----------- Secuencia:  0
----------- Paso:  1
----------- Secuencia:  0
----------- Paso:  2
Flat
----------- Secuencia:  1
----------- Paso:  0
----------- Secuencia:  1
----------- Paso:  1
----------- Secuencia:  1
----------- Paso:  2
Standing
----------- Secuencia:  2
----------- Paso:  0
----------- Secuencia:  2
----------- Paso:  1
----------- Secuencia:  2
----------- Paso:  2
Flat
----------- Secuencia:  3
----------- Paso:  0
----------- Secuencia:  3
----------- Paso:  1
----------- Secuencia:  3
----------- Paso:  2


##### Pruebas con pandas

In [91]:
df = pd.DataFrame(sim_obj.training_data)
df

,sequence,step,increments,prev_j_positions,prev_j_velocities,prev_j_forces,prev_base_pos_x,prev_base_pos_y,prev_base_pos_z,prev_base_ori_alpha,...,prev_base_ori_gamma,post_j_positions,post_j_velocities,post_j_forces,post_base_pos_x,post_base_pos_y,post_base_pos_z,post_base_ori_alpha,post_base_ori_beta,post_base_ori_gamma
0,0,0,"[0.12649222499953922, -0.02283292362893596, -0...","[-0.0010654926300048828, -0.04230475425720215,...","[-0.0061512053944170475, 0.0385761633515358, 0...","[-0.018592823296785355, 0.15814179182052612, 0...",0.000022,-0.000774,0.033502,0.000053,...,0.002250,"[0.12603044509887695, -0.10594701766967773, -0...","[0.035476718097925186, 0.11401187628507614, 0....","[-0.0008254386484622955, 0.12692295014858246, ...",0.002179,-0.001341,0.036847,0.003460,0.005102,0.025929
1,0,1,"[-0.117137965169735, -0.00830241518342736, -0....","[0.12603044509887695, -0.10594701766967773, -0...","[0.035476718097925186, 0.11401187628507614, 0....","[-0.0008254386484622955, 0.12692295014858246, ...",0.002179,-0.001341,0.036847,0.003460,...,0.025929,"[0.010022163391113281, -0.1569533348083496, -0...","[-0.009346017614006996, 0.05078320577740669, -...","[-0.03095586597919464, 0.19419336318969727, 0....",0.008719,-0.000843,0.036980,-0.003013,0.011240,0.019618
2,0,2,"[-0.0932235364555353, 0.15385486808366947, 0.1...","[0.010022163391113281, -0.1569533348083496, -0...","[-0.009346017614006996, 0.05078320577740669, -...","[-0.03095586597919464, 0.19419336318969727, 0....",0.008719,-0.000843,0.036980,-0.003013,...,0.019618,"[-0.08218502998352051, -0.04079580307006836, -...","[-0.01649858057498932, 0.2299787849187851, 0.0...","[0.023740509524941444, 0.13115733861923218, -0...",0.007281,0.002127,0.038514,-0.005605,0.052504,-0.005313
3,1,0,"[-0.06863673839126473, 0.08828472154303071, 0....","[-0.0010654926300048828, -0.04230475425720215,...","[-0.0061512053944170475, 0.0385761633515358, 0...","[-0.018592823296785355, 0.15814179182052612, 0...",0.000022,-0.000774,0.033502,0.000053,...,0.002250,"[-0.06345129013061523, 0.005017757415771484, 0...","[-0.17719285190105438, 0.23202918469905853, 0....","[0.1143677681684494, 0.11438461393117905, -0.1...",-0.003249,-0.003061,0.035764,0.004331,-0.001884,-0.009390
4,1,1,"[0.14790019061772586, -0.08850513981401781, -0...","[-0.06345129013061523, 0.005017757415771484, 0...","[-0.17719285190105438, 0.23202918469905853, 0....","[0.1143677681684494, 0.11438461393117905, -0.1...",-0.003249,-0.003061,0.035764,0.004331,...,-0.009390,"[0.07783985137939453, -0.11231708526611328, 0....","[0.3846172094345093, -0.1111508458852768, -0.1...","[-0.19164562225341797, 0.46284347772598267, 0....",-0.003244,-0.006890,0.037363,0.013865,-0.011680,-0.030703
5,1,2,"[-0.06253357761674555, -0.06083315250911609, 0...","[0.07783985137939453, -0.11231708526611328, 0....","[0.3846172094345093, -0.1111508458852768, -0.1...","[-0.19164562225341797, 0.46284347772598267, 0....",-0.003244,-0.006890,0.037363,0.013865,...,-0.030703,"[0.013831138610839844, -0.20002365112304688, 0...","[-0.011253368109464645, 0.009632119908928871, ...","[-0.03591739386320114, 0.5402714014053345, -0....",-0.005871,-0.008007,0.039180,0.013015,-0.002345,0.003956
6,2,0,"[-0.1293783406743649, 0.045238925987848434, -0...","[-0.0004067420959472656, 1.5587503910064697, 0...","[-0.46758660674095154, 0.39749154448509216, 0....","[-0.013305321335792542, 0.05467931181192398, -...",0.000006,-0.000882,0.117879,0.000037,...,-0.000120,"[-0.12984609603881836, 1.573868989944458, -0.0...","[-0.008964546956121922, 0.6238943338394165, -0...","[0.0007454557344317436, 1.1438368558883667, 0....",-0.000281,0.008243,0.118653,-0.003079,0.004715,0.034661
7,2,1,"[-0.11626931913177013, -0.1707370504235278, 0....","[-0.12984609603881836, 1.573868989944458, -0.0...","[-0.008964546956121922, 0.6238943338394165, -0...","[0.0007454557344317436, 1.1438368558883667, 0....",-0.000281,0.008243,0.118653,-0.003079,...,0.034661,"[-0.24599099159240723, 1.413752794265747, -0.0...","[-0.1076222509145

In [72]:
df.to_csv("data.csv", index=False)